# Elevated-Plus Maze Analysis

In [1]:
from analysis import analysis_utils as au
from IPython.core.interactiveshell import InteractiveShell
from multiprocessing import Process
from multiprocessing import Queue
import numpy as np
import os
import pandas as pd
import random
from scipy import stats
import seaborn as sns
import SigProc
import sys

In [2]:
%matplotlib inline
InteractiveShell.ast_node_interactivity = "all"

In [3]:
sns.set_style("darkgrid")

In [4]:
mouse_directory = os.path.expanduser("~") + "/Hen_Lab/Mice/EPM"

if not os.path.exists(mouse_directory):
    print("The mouse directory does not exist", file=sys.stderr)

file_num = 0
raw_files = list()
for dir_name, subdir_list, file_list in os.walk(mouse_directory):
    for file_name in file_list:
        if file_name.endswith(".csv"):
            print("{}. full path of: {} is: {}".format(file_num, file_name, dir_name+"/"+file_name))
            file_num += 1
            raw_files.append(dir_name+"/"+file_name)

0. full path of: behavior_drd87.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd87/behavior_drd87.csv
1. full path of: Raw_EPM_drd87.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd87/Raw_EPM_drd87.csv
2. full path of: Raw_EPM_drd73.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd73/Raw_EPM_drd73.csv
3. full path of: behavior_drd73.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd73/behavior_drd73.csv
4. full path of: Raw_EPM_drd77.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd77/Raw_EPM_drd77.csv
5. full path of: behavior_drd77.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd77/behavior_drd77.csv
6. full path of: behavior_drd46.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd46/behavior_drd46.csv
7. full path of: Raw_EPM_drd46.csv is: /Users/saveliyyusufov/Hen_Lab/Mice/EPM/drd46/Raw_EPM_drd46.csv


In [5]:
data = pd.read_csv(raw_files[1], header=None)
_, AUC_dataframe, cell_transients_dataframe = SigProc.detect_ca_transients_mossy(data, 2, 0.5, 0.2, 10)

In [6]:
# AUC_dataframe = au.downsample_dataframe(AUC_dataframe, 2)
# cell_transients_dataframe = au.downsample_dataframe(cell_transients_dataframe, 2)

In [7]:
au.neuron_line_plot(AUC_dataframe, "neuron6", "neuron15", "neuron19", "neuron28")

## TODO: Discuss and streamline the below functionality, turn it into a function, make sure the function is sound, and move it to `analysis_utils.py`

In [8]:
behavior_column_names = ['Trial_time', 'Recording_time', 'X_center', 'Y_center', 'Area', 'Areachange', 
                         'Elongation', 'Distance_moved', 'Velocity', 'Arena_centerpoint',
                         'Open1_centerpoint', 'Open2_centerpoint',
                         'Closed1_centerpoint', 'Closed2_centerpoint',
                         'OpenArms_centerpoint', 'ClosedArms_centerpoint', 'Result_1']

behavior_df = pd.read_csv(raw_files[5], header=None)
behavior_df.columns = behavior_column_names
behavior_df = au.downsample_dataframe(behavior_df, 3)

# Define what constitutes as a running frame
VELOCITY_CUTOFF = 4;

# Adds "Running_frames" column to the end of the behavior Dataframe 
behavior_df["Running_frames"] = np.where(behavior_df["Velocity"] > VELOCITY_CUTOFF, 1, 0)

In [9]:
neuron_concated_behavior = AUC_dataframe.join(behavior_df, how="left")

In [10]:
def compute_diff_rate(dataframe, neuron_activity_df, *behaviors, frame_rate=10):
    """Computes difference between the rates of two behaviors
    
    Args: 
        dataframe: the the concatenated pandas DataFrame of an animal's neuron 
        activity and corresponding behavior
        neuron_col_names: the names of the neuron columns in the DataFrame
        *behaviors: a single or ordered pair of behaviors to compute the difference
        rate for, e.g. "Running", e.g. "ClosedArms", "OpenArms"
        frame_rate: the framerate associated with the given data; default is 10
    
    Returns:
        a numpy array of all the means of the behavior vectors subtracted from the 
        corresponding means of the non-behavior vectors, all scaled by frame rate
    """
    if len(behaviors) == 1:  
        beh_vec = dataframe.loc[dataframe[behaviors[0]] != 0, neuron_activity_df.columns]
        no_beh_vec = dataframe.loc[dataframe[behaviors[0]] == 0, neuron_activity_df.columns]
        return frame_rate * (beh_vec.values.mean(axis=0) - no_beh_vec.values.mean(axis=0))
    elif len(behaviors) == 2:
        beh_vec = dataframe.loc[dataframe[behaviors[0]] != 0, neuron_activity_df.columns]
        no_beh_vec = dataframe.loc[dataframe[behaviors[1]] != 0, neuron_activity_df.columns]
        return frame_rate * (beh_vec.values.mean(axis=0) - no_beh_vec.values.mean(axis=0))

In [11]:
def set_real_diff_df(dataframe, neuron_act_df, behavior1, behavior2):
    """Compute the real difference mean values for all neurons
    
    Args:
        dataframe: the concatenated pandas DataFrame of the neuron activity
        DataFrame and corresponding behavior DataFrame, for a given animal
        neuron_activity_df: the pandas DataFrame of neuron activity,
        for a given animal
        behavior: the behavior for which to compute the difference rate
        
    
    Returns:
        real_df: a pandas DataFrame of with one row of all the real difference
        values computed for all the neurons for a given animal
    """
    real_df = pd.DataFrame(columns=neuron_act_df.columns, index=["d"])
    real_df.loc['d'] = compute_diff_rate(dataframe, neuron_act_df, behavior1, behavior2)
    return real_df

In [12]:
real_diff_df = set_real_diff_df(neuron_concated_behavior, AUC_dataframe, "OpenArms_centerpoint", "ClosedArms_centerpoint")
real_diff_df

,neuron1,neuron2,neuron3,neuron4,neuron5,neuron6,neuron7,neuron8,neuron9,neuron10,...,neuron60,neuron61,neuron62,neuron63,neuron64,neuron65,neuron66,neuron67,neuron68,neuron69
d,-3.59868,-1.40696,0.351025,-0.630193,-3.45533,-1.63303,-1.78319,-0.966683,0.131016,1.54682,...,-0.0513125,-1.28053,-0.355424,1.41495,2.6292,3.73675,-0.815443,0.142843,-0.242183,-0.00266278


In [13]:
def shuffle_worker(q, num_of_experiments, neuron_activity_df, neuron_and_behavior_df, behavior1, behavior2):
    """Helper function for shuffle()

    Given a certain number of experiments to simulate, this function will
    add a dataframe to a provided queue full of the amount of experiments 
    desired as obervations rows. 
    Note: This function is meant to be only be used as a helper function 
    for the shuffle() function

    Args:
        q: the blocking queue to which the resulting dataframe will be added to
        num_of_experiments: the number of experiments that will be simulated 
        and appended, as observations, to the dataframe to be returned
        neuron_activity_df: the neuron activity dataframe for a given mouse
        neuron_and_behavior_df: the concatenated neuron activity and behavior 
        dataframes for a given mouse 
        behavior: the specific behavior to simulate the experiments on
    """ 
    first_col = neuron_activity_df.columns[0]
    last_col = neuron_activity_df.columns[len(neuron_activity_df.columns)-1]
    shuffled_df = pd.DataFrame(columns=neuron_activity_df.columns)
    
    for index in range(num_of_experiments):
        neuron_and_behavior_df.loc[:, first_col:last_col] = neuron_and_behavior_df.loc[:, first_col:last_col].sample(frac=1).reset_index(drop=True)
        shuffled_df.loc[index] = compute_diff_rate(neuron_and_behavior_df, neuron_activity_df, behavior1, behavior2)

    q.put(shuffled_df)

In [14]:
def shuffle(total_experiments, neuron_and_behavior_df, neuron_activity_df, behavior1, behavior2):
    """Homebrewed resampling function for EPM Analysis
    
    Resampling function that gives the capability to "simulate"
    experiments using random shuffling of the observations for each 
    pandas dataframe. 
    
    Args: 
        total_experiments: the total amount of epxeriments to simulate via bootstrapping
        neuron_and_behavior_df: the concatenated neuron activity and behavior dataframes
        for a given animal
        neuron_activity_df: the neuron activity dataframe for a given animal
        behavior: the specific behavior to simulate the experiments on
    
    Returns: a (vertically) concatenated pandas DataFrame of all the shuffled DataFrames 
    that all the shuffle_worker processes produced
    """
    experiments_per_worker = total_experiments // os.cpu_count() 
    q = Queue()
    processes = []
    rets = []
    for _ in range(0, os.cpu_count()):
        p = Process(target=shuffle_worker, args=(q, experiments_per_worker, neuron_activity_df, neuron_and_behavior_df, behavior1, behavior2))
        processes.append(p)
        p.start()
    for p in processes:
        ret = q.get()  # will block
        rets.append(ret)
    for p in processes:
        p.join()

    return pd.concat(rets, ignore_index=True)

In [ ]:
import time

start = time.time()
resampled_df = shuffle(50000, neuron_concated_behavior, AUC_dataframe, "OpenArms_centerpoint", "ClosedArms_centerpoint")
end = time.time()
print(end-start)

In [84]:
import plotly
import plotly.graph_objs as go

x = resampled_df["neuron59"]
data = [go.Histogram(x=x, histnorm='probability')]
plotly.offline.iplot(data)

# Now, we compute a two-sided $p$-value as follows: $$\frac{1}{p} \sum_{i=1}^{p} \mathbb{1}(| \hat(D)_i | \ge |\hat(D)|)$$ for all of the neurons that were imaged

In [86]:
def compute_two_side_p_val(resampled_df, original_D_hat_df, neuron):
    return (1 / len(resampled_df.index)) * len(resampled_df.loc[:, neuron].loc[abs(resampled_df[neuron]) >= abs(original_D_hat_df[neuron].values[0])])

In [69]:
def get_num_of_events(dataframe, neuron):
    """Get the number of signal spikes for a given column vector
       
       Args:
           dataframe: a Pandas DataFrame that contains at least one 
           neuron's signal data, in column vector form.
           neuron: the name of the neuron column vector to get the 
           number of events for. 
       
       Returns:
           the amount of datapoints in a given column vector with 
           of nonzero value.
    """
    return len(dataframe.loc[:, neuron][dataframe[neuron] != 0])

## Now compute the amount of events that took place for each neuron. Neurons with less than $5$ transient events are labeled as "unclassifiable"

In [89]:
def classify_neurons(dataframe, resampled_df, p_value=0.05, threshold=10):
    """Classify neurons as selective or not-selective
    
        Use this function if your resampled data is NOT normally distributed. 
        Remember that this function can only tell you if a neuron is selective
        for a certain behavior or not. It will not classify what behavior the 
        behavior the neuron was actually selective for.

       Args:
           dataframe: a Pandas DataFrame that contains the neuron(s) column vector(s) to be classified.
           resampled_df: the Pandas Dataframe of all the computed rates after resampling.
           p_value: the cutoff value for the probability that an effect could occur by chance.
           threshold: the minimum required number of events for a neuron to be considered classifiable, 
           default is 10.

       Returns:
           classified_neurons: a dictionary of key-value pairs, where the name of each classified 
           neuron is a key, and that neuron's classification is the corresponding value. 
    """
    classified_neurons = dict()
    for neuron in dataframe.columns:
        if get_num_of_events(dataframe, neuron) < threshold:
            if not neuron in classified_neurons:
                classified_neurons[neuron] = "unclassified"
                
    for neuron in resampled_df.columns:
        if compute_two_side_p_val(resampled_df, real_diff_df, neuron) <= p_value:
            if not neuron in classified_neurons:
                classified_neurons[neuron] = "selective"
        else:
            if not neuron in classified_neurons:
                classified_neurons[neuron] = "not-selective"
                
    return classified_neurons

In [90]:
classify_neurons(AUC_dataframe, resampled_df, p_value=0.125, threshold=10)

{'neuron6': 'unclassified',
 'neuron15': 'unclassified',
 'neuron19': 'unclassified',
 'neuron28': 'unclassified',
 'neuron59': 'unclassified',
 'neuron60': 'unclassified',
 'neuron1': 'not-selective',
 'neuron2': 'not-selective',
 'neuron3': 'not-selective',
 'neuron4': 'not-selective',
 'neuron5': 'not-selective',
 'neuron7': 'not-selective',
 'neuron8': 'not-selective',
 'neuron9': 'not-selective',
 'neuron10': 'not-selective',
 'neuron11': 'not-selective',
 'neuron12': 'selective',
 'neuron13': 'not-selective',
 'neuron14': 'not-selective',
 'neuron16': 'not-selective',
 'neuron17': 'not-selective',
 'neuron18': 'not-selective',
 'neuron20': 'not-selective',
 'neuron21': 'not-selective',
 'neuron22': 'not-selective',
 'neuron23': 'not-selective',
 'neuron24': 'not-selective',
 'neuron25': 'selective',
 'neuron26': 'not-selective',
 'neuron27': 'selective',
 'neuron29': 'not-selective',
 'neuron30': 'not-selective',
 'neuron31': 'not-selective',
 'neuron32': 'not-selective',
 'neuro

In [91]:
for neuron in classified_neurons:
    if classified_neurons[neuron] == "selective":
        print(neuron)

neuron12
neuron25
neuron27
neuron35
neuron52
neuron53
neuron64
neuron65


## TODO: for `is_neuron_selective()`, make sure implementation is sound, write-up documentation, and move to analysis_utils.py

In [92]:
def two_tailed_neuron_classification(resampled_df, real_d_df, neuron, behavior_name, high_tail, low_tail):
    """Classifies a given neuron as selective or non-selective
    
    Use this function if your resampled data IS indeed normally distributed. 
    Classifies a given neuron as selective for a certain behavior, selective for 
    when that behavior is not performed, or non-selective. This is a custom function
    for carrying out a two-tailed hypothesis test.
    One can use this as a stand alone function to classify a single neuron for 
    a certain animal as either a <behavior> neuron, a "Not"-<behavior> neuron, or
    a "Non-selective" neuron.
    
    Args: 
        resampled_df: a resampled Pandas DataFrame with all the possible rates
        real_diff_df: a pandas DataFrame with one row that has the real difference of means
        values for a given animal and a corresponding behavior
        neuron: a single neuron of the neuron to classify use the 2-tailed hypothesis test
        behavior_name: the behavior to classify the neuron by, e.g. "Running" or "Non-Running"
        high_tail: the cutoff for the upper-tail of the distribution
        low_tail: the cutoff for the lower-tail of the distribution
    
    Returns:
        behavior_name, Not-<behavior_name>, or "Not-selective"; based on the result of the
        two-tailed hypothesis test. 
    """
    if real_d_df[neuron]['d'] >= np.percentile(resampled_df[neuron], high_tail):
        return behavior_name
    elif real_d_df[neuron]['d'] <= np.percentile(resampled_df[neuron], low_tail):
        return "Non-" + behavior_name
    else: 
        return "Not-selective"

In [94]:
is_neuron_selective(resampled_df, real_diff_df, "neuron60", "OpenArms", 87.5, 5)

'Not-selective'

In [81]:
def classify_neurons_for_beh(resampled_df, real_diff_df, behavior_name, high_tail, low_tail):
    """Classifies a given set of neurons
    
    This function simply calls is_neuron_selective for all the neurons 
    for a given animal. 
    
    Args: 
        resampled_df: a resampled Pandas DataFrame
        real_diff_df: a Pandas DataFrame with one row that has the real difference of means
        behavior_name: the behavior to classify each neuron by, e.g. "Running" or "Non-Running"
        high_tail: the cutoff for the upper-tail of the distribution
        low_tail: the cutoff for the lower-tail of the distribution
    
    Returns: 
        neurons_dict: a dictionary of all the neurons of a given animal as the keys,
        with each key having a corresponding classifcation as its value
    """
    neurons_dict = {}
    for neuron in resampled_df.columns:
        neurons_dict[neuron] = is_neuron_selective(resampled_df, real_diff_df, neuron, behavior_name, high_tail, low_tail)

    return neurons_dict

In [82]:
classify_neurons_for_beh(resampled_df, real_diff_df, "OpenArms_centerpoint", 87.5, 5)

{'neuron1': 'Not-selective',
 'neuron2': 'Not-selective',
 'neuron3': 'Not-selective',
 'neuron4': 'Not-selective',
 'neuron5': 'Non-OpenArms_centerpoint',
 'neuron6': 'Not-selective',
 'neuron7': 'Not-selective',
 'neuron8': 'Not-selective',
 'neuron9': 'Not-selective',
 'neuron10': 'Not-selective',
 'neuron11': 'Not-selective',
 'neuron12': 'OpenArms_centerpoint',
 'neuron13': 'Not-selective',
 'neuron14': 'Not-selective',
 'neuron15': 'Not-selective',
 'neuron16': 'Not-selective',
 'neuron17': 'Not-selective',
 'neuron18': 'Not-selective',
 'neuron19': 'Not-selective',
 'neuron20': 'Not-selective',
 'neuron21': 'Not-selective',
 'neuron22': 'Not-selective',
 'neuron23': 'Not-selective',
 'neuron24': 'Not-selective',
 'neuron25': 'OpenArms_centerpoint',
 'neuron26': 'Not-selective',
 'neuron27': 'OpenArms_centerpoint',
 'neuron28': 'Not-selective',
 'neuron29': 'Not-selective',
 'neuron30': 'Not-selective',
 'neuron31': 'Not-selective',
 'neuron32': 'Not-selective',
 'neuron33': 'Not

In [48]:
def activity_by_neurons(concated_df, neuron_names, *behaviors, frame_rate=10):
    """Computes the neuron activity rates for given behaviors
    
    This function computes the rates for a given animal's activity and  
    neuron, given some set of behaviors.

    Args: 
        concated_df: a concatenated pandas DataFrame of the neuron activity and 
        the corresponding behavior, for a given animal.
        neuron_names: the names of the neurons whose rates are to be computed.
        behaviors: a list of the behaviors for which to compute the activity rates. 
        frame_rate: the framerate to multiply the rate by, default is 10.

    Returns: 
        activity_df: a pandas DataFrame of the neuron activity rates.
    """
    activity_df = pd.DataFrame(columns=behaviors)
    
    for behavior in behaviors:
        if behavior in concated_df.columns:
            activity_df.loc[:, behavior] = frame_rate * concated_df.loc[concated_df[behavior] != 0, neuron_names].mean()
        elif '&' in behavior:
            beh1 = behavior.split('&')[0]
            beh2 = behavior.split('&')[1]
            activity_df.loc[:, behavior] = frame_rate * concated_df.loc[(concated_df[beh1] != 0) & ((concated_df[beh2] != 0)), neuron_names].mean()
        elif '|' in behavior:
            beh1 = behavior.split('|')[0]
            beh2 = behavior.split('|')[1]
            activity_df.loc[:, behavior] = frame_rate * concated_df.loc[(concated_df[beh1] != 0) | ((concated_df[beh2] != 0)), neuron_names].mean()

    return activity_df

In [49]:
activity_rates_df = activity_by_neurons(neuron_concated_behavior, AUC_dataframe.columns, "ClosedArms_centerpoint", "OpenArms_centerpoint", "OpenArms_centerpoint&Running_frames", "ClosedArms_centerpoint&Running_frames", "Running_frames")

## Plot scatterplot of neurons, where $x$-axis is the closedarms rate, $y$-axis is the openarms rate, and the entire scatterplot is bisected by the line $y=x$

In [50]:
classified_neurons

{'neuron6': 'unclassified',
 'neuron15': 'unclassified',
 'neuron19': 'unclassified',
 'neuron28': 'unclassified',
 'neuron59': 'unclassified',
 'neuron60': 'unclassified',
 'neuron1': 'not-selective',
 'neuron2': 'not-selective',
 'neuron3': 'not-selective',
 'neuron4': 'not-selective',
 'neuron5': 'not-selective',
 'neuron7': 'not-selective',
 'neuron8': 'not-selective',
 'neuron9': 'not-selective',
 'neuron10': 'not-selective',
 'neuron11': 'not-selective',
 'neuron12': 'selective',
 'neuron13': 'not-selective',
 'neuron14': 'not-selective',
 'neuron16': 'not-selective',
 'neuron17': 'not-selective',
 'neuron18': 'not-selective',
 'neuron20': 'not-selective',
 'neuron21': 'not-selective',
 'neuron22': 'not-selective',
 'neuron23': 'not-selective',
 'neuron24': 'not-selective',
 'neuron25': 'selective',
 'neuron26': 'not-selective',
 'neuron27': 'selective',
 'neuron29': 'not-selective',
 'neuron30': 'not-selective',
 'neuron31': 'not-selective',
 'neuron32': 'not-selective',
 'neuro

In [51]:
import plotly
import plotly.graph_objs as go

# Create traces

# Plot selective neurons
trace0 = go.Scatter(
    x = [activity_rates_df.loc[neuron, "OpenArms_centerpoint"] for neuron in classified_neurons if classified_neurons[neuron] == "selective"],
    y = [activity_rates_df.loc[neuron, "ClosedArms_centerpoint"] for neuron in classified_neurons if classified_neurons[neuron] == "selective"],
    text = [neuron for neuron in classified_neurons if classified_neurons[neuron] == "selective"],
    mode = "markers",
    name = "selective",
    marker = dict(
        color = 'rgb(255, 10, 193)'
    )
)

# Plot not-selective neurons
trace1 = go.Scatter(
    x = [activity_rates_df.loc[neuron, "OpenArms_centerpoint"] for neuron in classified_neurons if classified_neurons[neuron] == "not-selective"],
    y = [activity_rates_df.loc[neuron, "ClosedArms_centerpoint"] for neuron in classified_neurons if classified_neurons[neuron] == "not-selective"],
    text = [neuron for neuron in classified_neurons if classified_neurons[neuron] == "not-selective"],
    mode = "markers",
    name = "not-selective",
    marker = dict(
        color = 'rgb(105, 105, 105)'
    )
)

# Plot unclassified neurons
trace2 = go.Scatter(
    x = [activity_rates_df.loc[neuron, "OpenArms_centerpoint"] for neuron in classified_neurons if classified_neurons[neuron] == "unclassified"],
    y = [activity_rates_df.loc[neuron, "ClosedArms_centerpoint"] for neuron in classified_neurons if classified_neurons[neuron] == "unclassified"],
    text = [neuron for neuron in classified_neurons if classified_neurons[neuron] == "unclassified"],
    mode = "markers",
    name = "unclassified",
    marker = dict(
        color = "rgb(255, 69, 0)"
    )
)

# Plot y=x line
trace3 = go.Scatter(
    x = np.linspace(0, max(activity_rates_df["OpenArms_centerpoint"].max(), activity_rates_df["ClosedArms_centerpoint"].max())),
    y = np.linspace(0, max(activity_rates_df["OpenArms_centerpoint"].max(), activity_rates_df["ClosedArms_centerpoint"].max())),
    mode = 'lines',
    name = 'y=x'
)

data = [trace0, trace1, trace2, trace3]
layout = go.Layout(
    width=500,
    height=500,
    title="Open-arms neuron rates vs Closed-arms neuron rates",
)
fig = go.Figure(data=data, layout=layout)

plotly.offline.iplot(fig)

In [52]:
labels = ["selective", "non-selective", "unclassifiable"]
values = [sum(1 for classification in classified_neurons.values() if classification == "selective")]
values.append(sum(1 for classification in classified_neurons.values() if classification == "not-selective"))
values.append(sum(1 for classification in classified_neurons.values() if classification == "unclassified"))

trace = go.Pie(labels=labels, values=values)

plotly.offline.iplot([trace])

In [53]:
def behavior_by_time(concated_df, behavior):
    """Split dataframe by time and behavior
    
    Args: 
        concated_df: a concatenated pandas DataFrame of the neuron activity and 
        the corresponding behavior, for a given animal.
        behavior: the specific behavior for which to generate the time intervaled dataframes 
    
    Returns: 
        time_bins: a dictionary of each intervaled dataframe
    """
    
    # Create copy of the dataframe for a certain behavior
    time_binned_df = concated_df.loc[concated_df[behavior] != 0].copy()
    time_binned_df.reset_index(drop=True, inplace=True)
    
    # Add a column of the trial time in the form of time deltas
    x = pd.to_timedelta('0.1s')
    time_binned_df.loc[:, "TIME"] = pd.Series(x*i for i in (time_binned_df.index))
    
    # Group the dataframe by 1 minute intervals
    grouped = time_binned_df.set_index("TIME").groupby(pd.Grouper(freq="1Min"))
    
    # Place each dataframe that contains the data for every 1 minute intervals into a dictionary
    time_bins = {}
    minute = 0
    for name, group in grouped:
        time_bins[minute] = grouped.get_group(name)
        minute += 1
        
    return time_bins

In [ ]:
time_binned_dfs = behavior_by_time(neuron_concated_behavior, "OpenArms_centerpoint")
time_binned_dfs[0]